This code analyzes how the base distributions (given by p and q) in the paper changes according to 
difference frequency cutoffs and sentences lengths. I have made some modfications to the original source
code for this. Therefore, if you try this on the orginal code it won't work. 

In [1]:
from src.estimation import estimate_text_distribution
from src.MLE import MLE
import pandas as pd
# silence warnings
import warnings
warnings.filterwarnings("ignore")
import os 
import numpy as np

In [2]:
name = "ojs_ed"
human_data_21 = pd.read_parquet(f"data/training_data/{name}/human_data_21.parquet")
print(f"human_data_21: {human_data_21.shape[0]} sentences")
ai_data_21 = pd.read_parquet(f"data/training_data/{name}/ai_data_21.parquet")
print(f"ai_data_21: {ai_data_21.shape[0]} sentences")

human_data_21: 212521 sentences
ai_data_21: 451240 sentences


In [26]:
for ai_freq in range(1, 10, 2):
    for human_freq in range(1, 10, 2):
        for sent_len in range(1, 6):
            print("AI min freq: {}, human min freq: {}, sentence length: {}".format(ai_freq, 
                                                                                    human_freq, sent_len))
            estimate_text_distribution(f"data/training_data/{name}/human_data_21.parquet",
                                       f"data/training_data/{name}/ai_data_21.parquet",
                                       f"distribution_analysis/{name}_21_{ai_freq}_{human_freq}_{sent_len}.parquet", 
                                      min_ai=ai_freq, min_human=human_freq, min_sentence=sent_len)
            

AI min freq: 1, human min freq: 1, sentence length: 1
AI min freq: 1, human min freq: 1, sentence length: 2
AI min freq: 1, human min freq: 1, sentence length: 3
AI min freq: 1, human min freq: 1, sentence length: 4
AI min freq: 1, human min freq: 1, sentence length: 5
AI min freq: 1, human min freq: 3, sentence length: 1
AI min freq: 1, human min freq: 3, sentence length: 2
AI min freq: 1, human min freq: 3, sentence length: 3
AI min freq: 1, human min freq: 3, sentence length: 4
AI min freq: 1, human min freq: 3, sentence length: 5
AI min freq: 1, human min freq: 5, sentence length: 1
AI min freq: 1, human min freq: 5, sentence length: 2
AI min freq: 1, human min freq: 5, sentence length: 3
AI min freq: 1, human min freq: 5, sentence length: 4
AI min freq: 1, human min freq: 5, sentence length: 5
AI min freq: 1, human min freq: 7, sentence length: 1
AI min freq: 1, human min freq: 7, sentence length: 2
AI min freq: 1, human min freq: 7, sentence length: 3
AI min freq: 1, human min fr

In [43]:
# check how a sample distribution looks like 

sample_dist = pd.read_parquet(f"distribution_analysis/{name}_21_3_5_1.parquet")
display(sample_dist)
P = np.exp(sample_dist["logQ"].to_numpy())

,Word,logP,log1-P,logQ,log1-Q
0,delve,-9.868901,-0.000052,-5.937896,-0.002641
1,delves,-9.322357,-0.000089,-5.473278,-0.004206
2,shedding,-10.187355,-0.000038,-6.552492,-0.001428
3,underscore,-10.187355,-0.000038,-6.736685,-0.001187
4,encompass,-9.781889,-0.000056,-6.350124,-0.001748
...,...,...,...,...,...
12235,dalam,-6.417471,-0.001634,-11.407643,-0.000011
12236,yang,-6.238518,-0.001955,-11.407643,-0.000011
12237,lot,-6.646395,-0.001300,-11.918468,-0.000007
12238,meanwhile,-5.783689,-0.003082,-11.407643,-0.000011


18.58603886364394

In [3]:
def KL(P, Q):
    """ 
    computes KL divergence between P and Q
    Args: 
        P: (np.ndarray) 
        Q: (np.ndarray) 
    Return:
        (float) 
    """
    # normalize P and Q 
    P = P / np.sum(P)
    Q = Q / np.sum(Q)
    
    log_P = np.log(P) 
    log_Q = np.log(Q)
    pq = log_P/log_Q * P
    
    return np.sum(pq) 

In [49]:
for files in sorted(os.listdir(f"distribution_analysis")):
    if ".parquet" in files:
        details = files.split(".")[0].split("_")
        min_ai_freq = details[-3]
        min_human_freq = details[-2]
        min_sent_len = details[-1]
        print("Min ai freq: {}, min human freq: {}, min sent: {}".format(min_ai_freq, min_human_freq, 
                                                                        min_sent_len))
        df = pd.read_parquet(f"distribution_analysis/{files}")
        #print(df.shape, df.iloc[1000].to_numpy())
        KL1 = KL(np.exp(df["logP"].to_numpy()), np.exp(df["logQ"].to_numpy()))
        #KL2 = KL(df["log1-P"].to_numpy(), df["log1-Q"].to_numpy())
        print("Vocab size: {}".format(df.shape[0]))
        print("KL between P and Q: {}".format(KL1))
        print()

Min ai freq: 1, min human freq: 1, min sent: 1
Vocab size: 24731
KL between P and Q: 0.9815445898949204

Min ai freq: 1, min human freq: 1, min sent: 2
Vocab size: 24719
KL between P and Q: 0.9815327996482641

Min ai freq: 1, min human freq: 1, min sent: 3
Vocab size: 24701
KL between P and Q: 0.9815333003617038

Min ai freq: 1, min human freq: 1, min sent: 4
Vocab size: 24687
KL between P and Q: 0.9815585019637834

Min ai freq: 1, min human freq: 1, min sent: 5
Vocab size: 24661
KL between P and Q: 0.9816004382824437

Min ai freq: 1, min human freq: 3, min sent: 1
Vocab size: 16830
KL between P and Q: 0.9814942108579697

Min ai freq: 1, min human freq: 3, min sent: 2
Vocab size: 16816
KL between P and Q: 0.9814823942923743

Min ai freq: 1, min human freq: 3, min sent: 3
Vocab size: 16807
KL between P and Q: 0.9814831310495264

Min ai freq: 1, min human freq: 3, min sent: 4
Vocab size: 16790
KL between P and Q: 0.9815084090249515

Min ai freq: 1, min human freq: 3, min sent: 5
Vocab si